# Práctica 3. Reconocimiento de Entidades Nombradas. NER

Andrés González Flores

Procesamiento de Lenguaje Natural

Facultad de Ingeniería, UNAM

## Objetivo

Realizar un reconocimiento de Entidades Nombradas (NER) a partir de un modelo secuencial y con el corpus ‘ner dataset.csv’ que se proporciona.

## Instrucciones

Se deberán seguir los siguientes pasos:

1. Obtener las sentencias a partir del csv. En este archivo, se indica cada inicio de sentencia con ‘Sentence: n’. Se cuenta con 1000 sentencias que conformarán el corpus de entrenamiento y evaluación.
2. Preprocesar los datos.
3. Separar los datos en corpus de entrenamiento (70%) y corpus de evaluación (30%).
4. Entrenar un modelo secuencial a partir del corpus de entrenamiento. Deberán definirse los hiperparámetros.
5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.

## Dessarrollo

In [1]:
import csv
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import chain
from tqdm.notebook import tqdm as nbtqdm
from pprint import pprint

In [2]:
# Definición de constantes
SEED = 42
CORPUS_PATH = './ner_dataset.csv'
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<unk>'

np.random.seed(SEED)

### Paso 1. Obtener sentencias.


In [3]:
with open(CORPUS_PATH, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    csvlist = list(reader)

In [4]:
sentences = []
sent_i = []
for row in csvlist[1:]:
    if row[0] is not '':
        sentences.append(sent_i)
        sent_i = []
    sent_i.append((row[1], row[2]))
# Mi algoritmo está feo: agrega la primer sentencia vacía y no agrega la última
del sentences[0]
sentences.append(sent_i)

In [5]:
print(f'Número de oraciones = {len(sentences)}\n')
print('Ejemplos:')
print('\n'.join([' '.join([w for w, tag in sent]) for sent in sentences[:5] ]))

Número de oraciones = 1000

Ejemplos:
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "
They marched from the Houses of Parliament to a rally in Hyde Park .
Police put the number of marchers at 10000 while organizers claimed it was 1,00,000 .
The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .


### Paso 2. Preprocesamiento

In [6]:
freq_tokens = Counter([w for w,_ in chain(*sentences)])
freq_tokens.most_common(10)

[('the', 1111),
 ('.', 995),
 (',', 637),
 ('in', 576),
 ('of', 568),
 ('to', 505),
 ('a', 449),
 ('and', 391),
 ('The', 241),
 ("'s", 202)]

Agrego el diccionario de entrada

In [7]:
vocab_in = { 
    x[0] : i for i, x in enumerate(freq_tokens.most_common())
    if freq_tokens[x[0]] > 1
}

pprint(list(vocab_in.items())[:10])

[('the', 0),
 ('.', 1),
 (',', 2),
 ('in', 3),
 ('of', 4),
 ('to', 5),
 ('a', 6),
 ('and', 7),
 ('The', 8),
 ("'s", 9)]


Agrego los identificadores UNK, BOS Y EOS al vocabulario de entrada

In [8]:
vocab_in[UNK] = max(vocab_in.values())+1
vocab_in[BOS] = vocab_in[UNK]+1
vocab_in[EOS] = vocab_in[BOS]+1

pprint(list(vocab_in.items())[-10:])

[('collapse', 2102),
 ('Barno', 2103),
 ('reunification', 2104),
 ('Berlin', 2105),
 ('Kohl', 2106),
 ('proud', 2107),
 ('wall', 2108),
 ('<unk>', 2109),
 ('<BOS>', 2110),
 ('<EOS>', 2111)]


Agrego el vocabulario de salida (las etiquetas)

In [9]:
freq_tags = Counter([tag for _,tag in chain(*sentences)])
freq_tags.most_common()

[('', 18843),
 ('B-ge', 583),
 ('B-gpe', 543),
 ('B-rg', 413),
 ('I-per', 400),
 ('B-tim', 358),
 ('B-per', 327),
 ('I-rg', 290),
 ('I-ge', 105),
 ('I-tim', 77),
 ('B-art', 39),
 ('I-gpe', 26),
 ('I-art', 22),
 ('B-eve', 20),
 ('I-eve', 16),
 ('B-nat', 9),
 ('I-nat', 5)]

In [10]:
vocab_out = {
    y[0] : i for i, y in enumerate(freq_tags.most_common())
}
pprint(vocab_out)

{'': 0,
 'B-art': 10,
 'B-eve': 13,
 'B-ge': 1,
 'B-gpe': 2,
 'B-nat': 15,
 'B-per': 6,
 'B-rg': 3,
 'B-tim': 5,
 'I-art': 12,
 'I-eve': 14,
 'I-ge': 8,
 'I-gpe': 11,
 'I-nat': 16,
 'I-per': 4,
 'I-rg': 7,
 'I-tim': 9}


Creo los vocabularios inversos

In [11]:
vocab_in_inv = {v:k for k,v in vocab_in.items()}
pprint(list(vocab_in_inv.items())[::350])
print()

vocab_out_inv = {v:k for k,v in vocab_out.items()}
pprint(vocab_out_inv)

[(0, 'the'),
 (350, 'parliament'),
 (700, 'measure'),
 (1050, 'claim'),
 (1400, 'flown'),
 (1750, 'burning'),
 (2100, 'Tokar')]

{0: '',
 1: 'B-ge',
 2: 'B-gpe',
 3: 'B-rg',
 4: 'I-per',
 5: 'B-tim',
 6: 'B-per',
 7: 'I-rg',
 8: 'I-ge',
 9: 'I-tim',
 10: 'B-art',
 11: 'I-gpe',
 12: 'I-art',
 13: 'B-eve',
 14: 'I-eve',
 15: 'B-nat',
 16: 'I-nat'}


Creo un método que devuelve siempre un indice del vocabulario o el identificador UNK

In [12]:
def word_safe_vocab_index(w, vocab, ix_UNK):
    try:
        return vocab[w]
    except KeyError:
        return ix_UNK

Armo el corpus de entrenamiento

In [22]:
ix_UNK = vocab_in[UNK]
ix_BOS = vocab_in[BOS]
ix_EOS = vocab_in[EOS]

BOS_append = [(ix_BOS, vocab_out[''])] # BOS y su etiqueta
EOS_append = [(ix_EOS, vocab_out[''])] # EOS y su etiqueta

corpus = [
    BOS_append + [ # Agrego BOS y su etiqueta
        # Tupla con el valor del índice de la palabra y el índice de la etiqueta
        (
            word_safe_vocab_index(w, vocab_in, ix_UNK),
            vocab_out[tag]
        ) # Fin tupla
        for w, tag in sent # Para cada palabra en la sentencia
    ] + EOS_append # Fin append
    for sent in sentences # Por cada sentencia en la lista de sentencias
]
# pprint(corpus[:5])
print('\n\n'.join([' '.join([f'({ix_w}, {ix_tag})' for ix_w, ix_tag in sent]) for sent in corpus[:5] ]))

(2110, 0) (904, 0) (4, 0) (905, 0) (14, 0) (906, 0) (243, 0) (244, 1) (5, 0) (298, 0) (0, 0) (127, 0) (3, 0) (53, 1) (7, 0) (1282, 0) (0, 0) (679, 0) (4, 0) (99, 2) (77, 0) (17, 0) (16, 0) (71, 0) (1, 0) (2111, 0)

(2110, 0) (2109, 0) (4, 0) (177, 0) (31, 0) (3, 0) (0, 0) (680, 0) (551, 0) (0, 0) (458, 0) (59, 0) (459, 0) (2109, 0) (18, 0) (245, 0) (1283, 0) (25, 0) (28, 0) (200, 6) (2109, 0) (460, 0) (2109, 0) (28, 0) (7, 0) (28, 0) (2109, 0) (0, 0) (2109, 0) (1, 0) (28, 0) (2111, 0)

(2110, 0) (134, 0) (906, 0) (17, 0) (0, 0) (2109, 0) (4, 0) (2109, 0) (5, 0) (6, 0) (1284, 0) (3, 0) (2109, 1) (2109, 8) (1, 0) (2111, 0)

(2110, 0) (268, 0) (340, 0) (0, 0) (341, 0) (4, 0) (1285, 0) (22, 0) (1286, 0) (135, 0) (1287, 0) (299, 0) (32, 0) (19, 0) (2109, 0) (1, 0) (2111, 0)

(2110, 0) (8, 0) (298, 0) (552, 0) (15, 0) (0, 0) (1288, 0) (4, 0) (0, 0) (387, 0) (553, 0) (4, 0) (554, 1) (9, 0) (907, 0) (461, 3) (462, 7) (3, 0) (0, 0) (128, 0) (2109, 2) (2109, 0) (2109, 0) (4, 0) (2109, 1) (1, 0) 

### Paso 3. Separar corpus de entrenamiento y evaluacion

In [26]:
train_corpus, eval_corpus = train_test_split(corpus, test_size=0.3)
print("Ejemlos del corpus de entrenamiento")
for sent in train_corpus[:5]:
    print(sent)
print()
print("Ejemlos del corpus de evaluacion")
for sent in eval_corpus[:5]:
    print(sent)

Ejemlos del corpus de entrenamiento
[(2110, 0), (8, 0), (330, 2), (878, 0), (11, 0), (1221, 0), (22, 0), (6, 0), (2109, 0), (1994, 0), (2, 0), (2109, 0), (275, 0), (239, 0), (3, 0), (0, 0), (66, 0), (40, 5), (108, 0), (1, 0), (2111, 0)]
[(2110, 0), (119, 2), (49, 6), (302, 4), (372, 4), (7, 0), (2109, 0), (519, 3), (144, 6), (93, 4), (731, 4), (1484, 4), (64, 5), (699, 0), (0, 0), (98, 2), (54, 0), (9, 0), (2109, 0), (243, 0), (0, 0), (1807, 1), (656, 0), (764, 0), (1, 0), (2111, 0)]
[(2110, 0), (557, 2), (215, 0), (85, 0), (16, 0), (2045, 0), (17, 0), (0, 0), (115, 3), (247, 7), (1293, 7), (908, 7), (909, 7), (30, 0), (1165, 0), (5, 0), (86, 1), (141, 0), (258, 0), (169, 0), (1, 0), (2111, 0)]
[(2110, 0), (80, 3), (192, 7), (850, 0), (14, 0), (1709, 0), (961, 0), (188, 0), (892, 2), (41, 0), (92, 0), (0, 0), (1476, 0), (4, 0), (106, 0), (2092, 2), (144, 6), (93, 0), (2109, 6), (2093, 4), (1, 0), (2111, 0)]
[(2110, 0), (21, 2), (626, 6), (1056, 4), (1516, 4), (4, 0), (0, 0), (1517, 0),

In [27]:
# class Modelo(object):


4. Entrenar un modelo secuencial a partir del corpus de entrenamiento. Deberán definirse los hiperparámetros.
5. Evaluar el desempeño del sistema a partir del corpus de evaluación y con la métrica de Exactitud (Accuracy).
6. Ejemplificar el reconocimiento de entidades nombradas con 5 sentencias del corpus de evaluación.